In [1]:
# Importing required libraries

import pandas as pd
import numpy as np
import pickle
import random
import os
from glob import glob
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')
# To execute a cell line by line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Creating dictionary for DIAG ICD codes

In [2]:
lu_diag = pd.read_parquet('/N/project/optum/data/parquet/dodr_81_202207/lu_diagnosis.parquet', engine='pyarrow')
lu_diag.shape
lu_diag.head()

(332941, 10)

,DIAG_CD,DIAG_DESC,DIAG_FST3_CD,DIAG_FST3_DESC,DIAG_FST4_CD,DIAG_FST4_DESC,GDR_SPEC_CD,MDC_CD_DESC,MDC_CODE,ICD_VER_CD
0,A,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
1,X,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
2,1,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
3,20,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
4,28,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9


In [3]:
lu_diag[lu_diag['DIAG_DESC'] == 'UNKNOWN DIAGNOSIS']

,DIAG_CD,DIAG_DESC,DIAG_FST3_CD,DIAG_FST3_DESC,DIAG_FST4_CD,DIAG_FST4_DESC,GDR_SPEC_CD,MDC_CD_DESC,MDC_CODE,ICD_VER_CD
0,A,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
1,X,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
2,1,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
3,20,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
4,28,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
...,...,...,...,...,...,...,...,...,...,...
332936,ZZZZZ42,UNKNOWN DIAGNOSIS,ZZZ,UNKNOWN DIAGNOSIS,ZZZZ,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
332937,ZZZZZ4E,UNKNOWN DIAGNOSIS,ZZZ,UNKNOWN DIAGNOSIS,ZZZZ,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
332938,ZZZZZZ,UNKNOWN DIAGNOSIS,ZZZ,UNKNOWN DIAGNOSIS,ZZZZ,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
332939,ZZZZZZA,UNKNOWN DIAGNOSIS,ZZZ,UNKNOWN DIAGNOSIS,ZZZZ,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9


In [4]:
lu_diag[lu_diag['DIAG_DESC'] == 'NO DIAGNOSIS CODE']['DIAG_CD'].value_counts()

000      1
0000     1
00000    1
Name: DIAG_CD, dtype: int64

In [5]:
lu_diag[lu_diag['DIAG_DESC'] == 'UNKNOWN DIAGNOSIS']['DIAG_CD'].value_counts()

A          7
5          6
N          6
0          6
X          6
          ..
77808      1
77809      1
77810      1
77811      1
ZZZZZZZ    1
Name: DIAG_CD, Length: 210719, dtype: int64

In [6]:
lu_diag[['DIAG_CD', 'DIAG_DESC', 'ICD_VER_CD']]

,DIAG_CD,DIAG_DESC,ICD_VER_CD
0,A,UNKNOWN DIAGNOSIS,10
1,X,UNKNOWN DIAGNOSIS,10
2,1,UNKNOWN DIAGNOSIS,9
3,20,UNKNOWN DIAGNOSIS,9
4,28,UNKNOWN DIAGNOSIS,9
...,...,...,...
332936,ZZZZZ42,UNKNOWN DIAGNOSIS,10
332937,ZZZZZ4E,UNKNOWN DIAGNOSIS,10
332938,ZZZZZZ,UNKNOWN DIAGNOSIS,9
332939,ZZZZZZA,UNKNOWN DIAGNOSIS,9


In [7]:
lu_diag = lu_diag[~lu_diag['DIAG_DESC'].isin(['UNKNOWN DIAGNOSIS', 'NO DIAGNOSIS CODE'])]
lu_diag.shape

(113459, 10)

In [8]:
ids = lu_diag["DIAG_CD"]
lu_diag[ids.isin(ids[ids.duplicated()])].sort_values("DIAG_CD")

,DIAG_CD,DIAG_DESC,DIAG_FST3_CD,DIAG_FST3_DESC,DIAG_FST4_CD,DIAG_FST4_DESC,GDR_SPEC_CD,MDC_CD_DESC,MDC_CODE,ICD_VER_CD
127474,C811,NODULAR SCLEROSIS HODGKIN LYMPHOMA,C81,HODGKIN LYMPHOMA,C811,NOD SCLEROS CLASS HODGKIN LYMPHOMA,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
127475,C811,NODULAR SCLEROSIS HODGKIN LYMPHOMA,C81,UNKNOWN DIAGNOSIS,C811,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
127476,C8110,NOD SCLER HODGKIN LYMPHOMA UNS SITE,C81,HODGKIN LYMPHOMA,C811,NOD SCLEROS CLASS HODGKIN LYMPHOMA,,"MYELOPROLIFERATIVE DISEASES & DISORDERS, POORLY D",17,10
127477,C8110,NOD SCLER HODGKIN LYMPHOMA UNS SITE,C81,UNKNOWN DIAGNOSIS,C811,UNKNOWN DIAGNOSIS,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
127489,C812,MIXED CELLULARITY HODGKIN LYMPHOMA,C81,HODGKIN LYMPHOMA,C812,MC CLASSICAL HODGKIN LYMPHOMA,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
...,...,...,...,...,...,...,...,...,...,...
312266,V9121,QUAD GEST 2/> MONOCHORIONIC FETUSES,V91,MULTIPLE GESTATION PLACENTA STATUS,V912,QUADRUPLET GESTATION PLACENTA STS,,FACTORS INFLUENCING HEALTH STATUS & OTHER CONTACT,23,9
312270,V9122,FALL COLL BETW FISH BOAT OTH WC/OBJ,V91,OTH INJURY D/T ACCIDENT WATERCRAFT,V912,FALL COLL BETWN WATRCRFT OTH WC/OBJ,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
312271,V9122,QUAD GEST 2/> MONOAMNIOTIC FETUSES,V91,MULTIPLE GESTATION PLACENTA STATUS,V912,QUADRUPLET GESTATION PLACENTA STS,,FACTORS INFLUENCING HEALTH STATUS & OTHER CONTACT,23,9
312291,V9129,FALL COLL BETWN UNS WATRCRFT WC/OBJ,V91,OTH INJURY D/T ACCIDENT WATERCRAFT,V912,FALL COLL BETWN WATRCRFT OTH WC/OBJ,,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10


In [10]:
lu_diag[lu_diag['DIAG_CD'] == '001']

,DIAG_CD,DIAG_DESC,DIAG_FST3_CD,DIAG_FST3_DESC,DIAG_FST4_CD,DIAG_FST4_DESC,GDR_SPEC_CD,MDC_CD_DESC,MDC_CODE,ICD_VER_CD
2322,001,CHOLERA,001,CHOLERA,,,,DISEASES & DISORDERS OF THE DIGESTIVE SYSTEM,06,9


In [12]:
diag_df = lu_diag[['DIAG_CD', 'DIAG_DESC']]
diag_df = diag_df.drop_duplicates()
diag_df.shape
diag_df.head()

(113420, 2)

,DIAG_CD,DIAG_DESC
2322,001,CHOLERA
2328,0010,CHOLERA DUE TO VIBRIO CHOLERAE
2345,0011,CHOLERA DUE VIBRIO CHOLERAE EL TOR
2446,0019,UNSPECIFIED CHOLERA
2458,002,TYPHOID AND PARATYPHOID FEVERS


In [13]:
diag_df.isnull().sum()

DIAG_CD      0
DIAG_DESC    0
dtype: int64

In [14]:
diag_df[diag_df['DIAG_CD'] == '001']

,DIAG_CD,DIAG_DESC
2322,001,CHOLERA


In [15]:
diag_df[diag_df['DIAG_CD'] == '0010']

,DIAG_CD,DIAG_DESC
2328,0010,CHOLERA DUE TO VIBRIO CHOLERAE


In [16]:
diag_df.to_csv('Opioid_analytics/Akhila/overdose_prediction/files/diag_icd_codes.csv', index=False)

# Creating dictionary for PROC codes

In [17]:
lu_proc = pd.read_parquet('/N/project/optum/data/parquet/dodr_81_202207/lu_procedure.parquet', engine='pyarrow')
lu_proc.shape

(340926, 8)

In [18]:
lu_proc[lu_proc['PROC_DESC'] == 'UNKNOWN PROCEDURE']

,CATEGORY_DTL_CD,CATEGORY_DTL_CODE_DESC,CATEGORY_GENL_CD,CATEGORY_GENL_CODE_DESC,PROC_CD,PROC_DESC,PROC_END_DATE,PROC_TYP_CD
0,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,0,UNKNOWN PROCEDURE,9999-12-31,UNKN
1,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,2,UNKNOWN PROCEDURE,9999-12-31,UNKN
2,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,0,UNKNOWN PROCEDURE,9999-12-31,UNKN
3,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,1,UNKNOWN PROCEDURE,9999-12-31,UNKN
4,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,2,UNKNOWN PROCEDURE,9999-12-31,UNKN
...,...,...,...,...,...,...,...,...
340921,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,ZZZZ3,UNKNOWN PROCEDURE,9999-12-31,UNKN
340922,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,ZZZZ5,UNKNOWN PROCEDURE,9999-12-31,UNKN
340923,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,ZZZZJ,UNKNOWN PROCEDURE,9999-12-31,UNKN
340924,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,ZZZZX,UNKNOWN PROCEDURE,9999-12-31,UNKN


In [19]:
lu_proc[lu_proc['PROC_DESC'] == 'NO PROCEDURE CODE']['PROC_CD'].value_counts()

0000     1
00000    1
Name: PROC_CD, dtype: int64

In [20]:
lu_proc[lu_proc['PROC_DESC'] == 'UNKNOWN PROCEDURE']['PROC_CD'].value_counts()

2        6
0        5
9        5
N        4
D        4
        ..
66831    1
66832    1
66833    1
66836    1
ZZZZZ    1
Name: PROC_CD, Length: 225349, dtype: int64

In [21]:
lu_proc[['PROC_CD', 'PROC_DESC']]

,PROC_CD,PROC_DESC
0,0,UNKNOWN PROCEDURE
1,2,UNKNOWN PROCEDURE
2,0,UNKNOWN PROCEDURE
3,1,UNKNOWN PROCEDURE
4,2,UNKNOWN PROCEDURE
...,...,...
340921,ZZZZ3,UNKNOWN PROCEDURE
340922,ZZZZ5,UNKNOWN PROCEDURE
340923,ZZZZJ,UNKNOWN PROCEDURE
340924,ZZZZX,UNKNOWN PROCEDURE


In [22]:
lu_proc = lu_proc[~lu_proc['PROC_DESC'].isin(['UNKNOWN PROCEDURE', 'NO PROCEDURE CODE'])]
lu_proc.shape

(112767, 8)

In [23]:
ids = lu_proc["PROC_CD"]
lu_proc[ids.isin(ids[ids.duplicated()])].sort_values("PROC_CD")

,CATEGORY_DTL_CD,CATEGORY_DTL_CODE_DESC,CATEGORY_GENL_CD,CATEGORY_GENL_CODE_DESC,PROC_CD,PROC_DESC,PROC_END_DATE,PROC_TYP_CD
5431,236,NOT ASSIGNED,16,"MISC DIAG, THERAPEUTIC",0045T,WHOLE BDY INTEG PHOTOHX DYSPLASTC NEVI/MELANOMA,2008-01-31,UNKN
5432,174,OTH NON-OR THER PROC SKIN/BREA,15,OPER INTEGUMENTARY SYS,0045T,WHOLE BODY PHOTOGRAPHY,9999-12-31,CPT-4
5448,236,NOT ASSIGNED,16,"MISC DIAG, THERAPEUTIC",0046T,CATH LAVAGE MAMM DUCTHI RSK IND EA BRST; 1 DUCT,2003-12-31,UNKN
5449,165,BREAST BIOPSY AND OTHER DIAGNO,15,OPER INTEGUMENTARY SYS,0046T,CATH LAVGE MAMM DUCT EA BRST,9999-12-31,CPT-4
5541,236,NOT ASSIGNED,16,"MISC DIAG, THERAPEUTIC",0051T,IMPL TOTAL REPL HEARTSYS W/RECIPIENT CARDIECT,2007-02-28,UNKN
...,...,...,...,...,...,...,...,...
318820,240,MEDICATIONS,16,MISC DIAG THERAPEUTIC,T1502,ADMN ORL IM&/SUBQ MED HLTH PROF,9999-12-31,HCPCS
318936,237,ANCILLARY SERVICES,16,MISC DIAG THERAPEUTIC,T2010,PASRR LEVEL I ID SCREEN PER SCREEN,9999-12-31,HCPCS
318937,236,NOT ASSIGNED,16,"MISC DIAG, THERAPEUTIC",T2010,PASRR LEVL I IDENTIFICATION SCREEN PER SCREEN,2004-03-31,UNKN
318938,237,ANCILLARY SERVICES,16,MISC DIAG THERAPEUTIC,T2011,PASRR LEVEL II EVALUATION PER EVAL,9999-12-31,HCPCS


In [24]:
lu_proc[lu_proc['PROC_CD'] == '0045T']

,CATEGORY_DTL_CD,CATEGORY_DTL_CODE_DESC,CATEGORY_GENL_CD,CATEGORY_GENL_CODE_DESC,PROC_CD,PROC_DESC,PROC_END_DATE,PROC_TYP_CD
5431,236,NOT ASSIGNED,16,"MISC DIAG, THERAPEUTIC",0045T,WHOLE BDY INTEG PHOTOHX DYSPLASTC NEVI/MELANOMA,2008-01-31,UNKN
5432,174,OTH NON-OR THER PROC SKIN/BREA,15,OPER INTEGUMENTARY SYS,0045T,WHOLE BODY PHOTOGRAPHY,9999-12-31,CPT-4


In [25]:
lu_proc[lu_proc['PROC_CD'] == '0046T']

,CATEGORY_DTL_CD,CATEGORY_DTL_CODE_DESC,CATEGORY_GENL_CD,CATEGORY_GENL_CODE_DESC,PROC_CD,PROC_DESC,PROC_END_DATE,PROC_TYP_CD
5448,236,NOT ASSIGNED,16,"MISC DIAG, THERAPEUTIC",0046T,CATH LAVAGE MAMM DUCTHI RSK IND EA BRST; 1 DUCT,2003-12-31,UNKN
5449,165,BREAST BIOPSY AND OTHER DIAGNO,15,OPER INTEGUMENTARY SYS,0046T,CATH LAVGE MAMM DUCT EA BRST,9999-12-31,CPT-4


In [26]:
# dropping duplicates to keep only the most recent codes

lu_proc = lu_proc.drop_duplicates('PROC_CD',keep='last')

In [27]:
proc_df = lu_proc[['PROC_CD', 'PROC_DESC']]
proc_df = proc_df.drop_duplicates()
proc_df.shape
proc_df.head()

(112571, 2)

,PROC_CD,PROC_DESC
4115,.2540,AUDIO DIAGNOSTIC BATTERY
4164,00,PROCEDURES AND INTERVENTIONS NEC
4174,000,THERAPEUTIC ULTRASOUND
4222,0001,THERAPEUTIC US VESSELS HEAD&NECK
4234,0001A,ADM SARSCOV2 30MCG/0.3ML 1ST


In [28]:
proc_df.isnull().sum()

PROC_CD      0
PROC_DESC    0
dtype: int64

In [29]:
proc_df[proc_df['PROC_CD'] == '0046T']

,PROC_CD,PROC_DESC
5449,0046T,CATH LAVGE MAMM DUCT EA BRST


In [30]:
proc_df[proc_df['PROC_CD'] == '0045T']

,PROC_CD,PROC_DESC
5432,0045T,WHOLE BODY PHOTOGRAPHY


In [37]:
proc_df[proc_df['PROC_CD'] == '3893']

,PROC_CD,PROC_DESC
131439,3893,VENOUS CATHETERIZATION NEC


In [32]:
proc_df.to_csv('Opioid_analytics/Akhila/overdose_prediction/files/proc_codes.csv', index=False)